In [1]:
import os
os.chdir("../")
%pwd


'c:\\Users\\IT STORE\\Desktop\\ML-Projects\\Text-Summarizer'

In [2]:
# from transformers import TrainingArguments

# # # Define training arguments
# trainer_args = TrainingArguments(
#     output_dir="./outputs",  # Output directory for checkpoints, tensorboard logs, etc.
#     overwrite_output_dir=True,  # Overwrite existing output directory (optional)
#     num_train_epochs=3,  # Number of training epochs
#     per_device_train_batch_size=4,  # Batch size per training GPU/TPU
#     save_steps=10_000,  # Save model checkpoint every N steps
#     save_total_limit=2,  # Maximum number of checkpoints to save
#     logging_steps=500,  # Log training information every N steps
#     evaluation_strategy="epoch",  # Evaluate model performance after each epoch
#     metric_for_best_model="loss",  # Metric to monitor for early stopping (minimize loss)
#     load_best_model_at_end=True,  # Load the best model based on the monitored metric
#     warmup_steps=500,  # Number of warmup steps for learning rate scheduler
#     learning_rate=2e-5,  # Learning rate for the optimizer
#     adam_epsilon=1e-8,  # Epsilon for Adam optimizer
#     weight_decay=0.01,  # Weight decay for optimizer
# )

# Feel free to modify these arguments based on your specific needs

# # Example usage with transformers.Trainer
# from transformers import Trainer, BartForConditionalGeneration

# model_name = "facebook/bart-large-cnn"
# tokenizer = BartTokenizer.from_pretrained(model_name)
# model = BartForConditionalGeneration.from_pretrained(model_name)

# # Prepare your training data (text and labels)
# train_data = ...  # Your training dataset

# trainer = Trainer(
#     model=model,
#     args=trainer_args,
#     tokenizer=tokenizer,
#     train_dataset=train_data,
# )

# # Start training
# trainer.train()


In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    overwrite_output_dir: bool
    num_train_epochs: int # Number of training epochs
    per_device_train_batch_size: int  # Batch size per training GPU/TPU
    save_steps: int   # Save model checkpoint every N steps
    save_total_limit: int  # Maximum number of checkpoints to save
    logging_steps: int # Log training information every N steps
    evaluation_strategy: str  # Evaluate model performance after each epoch
    metric_for_best_model: str  # Metric to monitor for early stopping (minimize loss)
    load_best_model_at_end: bool  # Load the best model based on the monitored metric
    warmup_steps: int  # Number of warmup steps for learning rate scheduler
    learning_rate: float  # Learning rate for the optimizer
    adam_epsilon: float  # Epsilon for Adam optimizer
    weight_decay: float  # Weight decay for optimizer


In [4]:
from src.constants import *
from src.utils import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self):
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        create_directories([config.root_dir])
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            overwrite_output_dir = config.overwrite_output_dir,
            num_train_epochs = params.num_train_epochs,
            per_device_train_batch_size= config.per_device_train_batch_size,
            save_steps = config.save_steps,
            save_total_limit = config.save_total_limit,
            logging_steps = config.logging_steps,
            evaluation_strategy = config.evaluation_strategy,
            metric_for_best_model = config.metric_for_best_model,
            load_best_model_at_end = config.load_best_model_at_end,
            warmup_steps = config.warmup_steps,
            learning_rate = config.learning_rate,
            adam_epsilon = config.adam_epsilon,
            weight_decay = config.weight_decay
        )
        
        return model_trainer_config

In [6]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import BartForConditionalGeneration, BertTokenizer
from datasets import load_dataset, load_from_disk
import torch

[2024-06-08 20:44:22,038: WARNING: module_wrapper: From c:\Users\IT STORE\anaconda3\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]
[2024-06-08 20:44:22,232: INFO: utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-06-08 20:44:22,233: INFO: utils: NumExpr defaulting to 8 threads.]


In [7]:
from transformers import TrainingArguments, Trainer
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import DataCollatorForSeq2Seq
from datasets import load_from_disk
import torch
import os

class ModelTrainer:
    def __init__(self, config):
        self.config = config
        
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"  # Corrected to torch.cuda.is_available()
        tokenizer = BartTokenizer.from_pretrained(self.config.model_ckpt)  # Assuming BartTokenizer is correct
        model_bert = BartForConditionalGeneration.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_bert)
        
        # loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir,
        #     overwrite_output_dir=True,
        #     num_train_epochs=self.config.num_train_epochs,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size,
        #     save_steps=self.config.save_steps,
        #     save_total_limit=self.config.save_total_limit,
        #     logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy,
        #     metric_for_best_model=self.config.metric_for_best_model,
        #     load_best_model_at_end=self.config.load_best_model_at_end,
        #     warmup_steps=self.config.warmup_steps,
        #     learning_rate=self.config.learning_rate,
        #     adam_epsilon=self.config.adam_epsilon,
        #     weight_decay=self.config.weight_decay
        # )
        
        trainer_args=  TrainingArguments(
            output_dir=self.config.root_dir,  # Output directory for checkpoints, tensorboard logs, etc.
            overwrite_output_dir=True,  # Overwrite existing output directory (optional)
            num_train_epochs=3,  # Number of training epochs
            per_device_train_batch_size=4,  # Batch size per training GPU/TPU
            save_steps=10_000,  # Save model checkpoint every N steps
            save_total_limit=2,  # Maximum number of checkpoints to save
            logging_steps=500,  # Log training information every N steps
            evaluation_strategy="epoch",  # Evaluate model performance after each epoch
            metric_for_best_model="loss",  # Metric to monitor for early stopping (minimize loss)
            load_best_model_at_end=True,  # Load the best model based on the monitored metric
            warmup_steps=500,  # Number of warmup steps for learning rate scheduler
            learning_rate=2e-5,  # Learning rate for the optimizer
            adam_epsilon=1e-8,  # Epsilon for Adam optimizer
            weight_decay=0.01,  # Weight decay for optimizer
        )

        
        model_trainer = Trainer(
            model=model_bert,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["train"],
            eval_dataset=dataset_samsum_pt["test"],
        )
        
        model_trainer.train()
        
        # Save model and tokenizer
        model_dir = os.path.join(self.config.root_dir, "bart-large-cnn-model")
        tokenizer_dir = os.path.join(self.config.root_dir, "bart-large-cnn-tokenizer")
        
        # Ensure the directories exist
        os.makedirs(model_dir, exist_ok=True)
        os.makedirs(tokenizer_dir, exist_ok=True)
        
        model_bert.save_pretrained(model_dir)
        tokenizer.save_pretrained(tokenizer_dir)


In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    print(e)

[2024-06-08 20:44:24,436: INFO: utils: yaml file: config\config.yaml loaded successfully.]
[2024-06-08 20:44:24,440: INFO: utils: yaml file: params.yaml loaded successfully.]
[2024-06-08 20:44:24,442: INFO: utils: Created directory at: artifacts]
[2024-06-08 20:44:24,443: INFO: utils: Created directory at: artifacts/data_trainer]
"'ConfigBox' object has no attribute 'overwrite_output_dir'"
